In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.tree import export_graphviz
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from matplotlib import font_manager, rc
from matplotlib import style

import numpy as np
from scipy import stats
import scipy.stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import shapiro 
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
matplotlib.rc('font',family="NanumGothic")

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os

from sklearn.tree import export_graphviz
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

from sklearn.tree import export_graphviz

os.environ["PATH"]+=os.pathsep+"~"
matplotlib.rc('font',family="NanumGothic")

#### 데이터 구성 확인

In [2]:
df_order=pd.read_csv("log_order_sheet_001.csv")
df_member=pd.read_csv("master_member_sheet_001.csv")
df_product=pd.read_csv("master_product_sheet_002.csv")
df_cat=pd.read_csv("modifed_log_order_with_category.csv")
df_cat["영업이익"]=df_order["상품구매금액"]-df_order["공급원가"]*df_order["주문수량"]
df_order["영업이익"]=df_order["상품구매금액"]-df_order["공급원가"]*df_order["주문수량"]
df_order[:-1].head(5)

/home/piai/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,결제수단,공급원가,교환구분,네이버 포인트,배송번호,배송비 정보,배송시작일,배송완료일,사용한 적립금액,...,카드사,쿠폰 할인금액,판매가,품목번호,친환경,사은품,회원,주문수량,총수량,영업이익
0,0,무통장입금,44550.0,교환안함,0,D-20181227-0000648-00,무료,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,...,기타,0,71450,42602,False,False,True,1,300,26900.0
1,1,무통장입금,44000.0,교환안함,0,D-20181229-0000119-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,기타,0,70620,42930,False,False,True,2,2000,53240.0
2,2,무통장입금,6820.0,교환안함,0,D-20181230-0000100-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,기타,0,13910,42998,False,False,False,1,200,7090.0
3,3,무통장입금,49500.0,교환안함,0,D-20181231-0000087-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,기타,0,81600,43063,False,False,True,1,300,32100.0
4,4,무통장입금,61600.0,교환안함,0,D-20181231-0000108-00,무료,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,...,기타,0,94710,43066,False,False,True,1,400,33110.0


# ★ 의사결정나무 (x인자(나이,성별)에 따른 y(구매금액))

In [3]:
df_order.columns

Index(['Unnamed: 0', '결제수단', '공급원가', '교환구분', '네이버 포인트', '배송번호', '배송비 정보',
       '배송시작일', '배송완료일', '사용한 적립금액', '상품구매금액', '상품명', '상품번호', '상품별 추가할인금액',
       '수령인 주소', '주문경로', '주문번호', '주문일시', '주문자ID', '카드사', '쿠폰 할인금액', '판매가',
       '품목번호', '친환경', '사은품', '회원', '주문수량', '총수량', '영업이익'],
      dtype='object')

In [4]:
df_product.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', '#', '공급사', '과세/면세', '담기', '매입가',
       '상품명(기본)', '상품코드', '재입고알림', '카테고리', '판매가', '품절여부', '후기', '분류카테고리'],
      dtype='object')

In [5]:
df_member.columns

Index(['Unnamed: 0', 'ID', '나이', '사용가능 적립금', '성별', '주소(동/읍/면)', '최종접속일',
       '최종주문일', '유입경로', '유입기기', '회원 가입일', '사업자구분', '회원구분'],
      dtype='object')

In [6]:
df_new1=pd.pivot_table(data=df_order,index="주문자ID",values='상품구매금액').reset_index()
df_new1.head(3)

,주문자ID,상품구매금액
0,100304734@n,103300.000000
1,1003409866@k,95641.777778
2,100381931@n,23683.333333


In [73]:
df_new1=df_new1.rename({'주문자ID':'ID','영업이익':'영업이익'},axis='columns')

In [74]:
df_result1=pd.merge(df_new1,df_member,on='ID')
df_result1.head(5)

,ID,상품구매금액,Unnamed: 0,나이,사용가능 적립금,성별,주소(동/읍/면),최종접속일,최종주문일,유입경로,유입기기,회원 가입일,사업자구분,회원구분
0,100304734@n,103300.000000,6308,40대,100,남자,주소없음,2019-05-09 오후 2:09,2019-05-09 오후 2:12,SNS광고,모바일,2019-05-09,개인,일반
1,1003409866@k,95641.777778,4442,20대,55900,남자,서울특별시 강남구 광평로51길 27 (수서동),2019-05-28 오후 4:40,2019-05-28 오후 4:41,카카오톡광고,모바일,2019-01-07,개인,일반
2,100381931@n,23683.333333,3587,40대,1000,남자,대구광역시 남구 대봉로 183 (이천동),2019-02-10 오후 9:37,2019-02-10 오후 9:46,SNS광고,모바일,2018-10-31,개인,일반
3,1004498382@k,76400.000000,4492,40대,1300,남자,경기도 부천시 부천로476번길 46 (오정동),2019-01-09 오후 12:30,2019-01-09 오후 12:49,SNS광고,모바일,2019-01-09,개인,일반
4,1004547839@k,180400.000000,4493,40대,2400,남자,전라남도 나주시 그린로 317-7 (빛가람동),2019-02-14 오후 2:08,2019-01-09 오후 1:48,블로그광고,PC,2019-01-09,개인,일반


#### 1, 나이 연속현 변경

In [89]:
df_result1=df_result1.replace({'나이':'20대'},{'나이':'20'})
df_result1=df_result1.replace({'나이':'30대'},{'나이':'30'})
df_result1=df_result1.replace({'나이':'40대'},{'나이':'40'})
df_result1=df_result1.replace({'나이':'50대'},{'나이':'50'})
df_result1=df_result1.replace({'나이':'60대'},{'나이':'60'})
df_result1=df_result1.replace({'나이':'70대'},{'나이':'70'})
df_result1=df_result1.replace({'나이':'80대'},{'나이':'80'})
df_result1.head(10)

,ID,상품구매금액,Unnamed: 0,나이,사용가능 적립금,성별,주소(동/읍/면),최종접속일,최종주문일,유입경로,유입기기,회원 가입일,사업자구분,회원구분
0,100304734@n,103300.000000,6308,40,100,남자,주소없음,2019-05-09 오후 2:09,2019-05-09 오후 2:12,SNS광고,모바일,2019-05-09,개인,일반
1,1003409866@k,95641.777778,4442,20,55900,남자,서울특별시 강남구 광평로51길 27 (수서동),2019-05-28 오후 4:40,2019-05-28 오후 4:41,카카오톡광고,모바일,2019-01-07,개인,일반
2,100381931@n,23683.333333,3587,40,1000,남자,대구광역시 남구 대봉로 183 (이천동),2019-02-10 오후 9:37,2019-02-10 오후 9:46,SNS광고,모바일,2018-10-31,개인,일반
3,1004498382@k,76400.000000,4492,40,1300,남자,경기도 부천시 부천로476번길 46 (오정동),2019-01-09 오후 12:30,2019-01-09 오후 12:49,SNS광고,모바일,2019-01-09,개인,일반
4,1004547839@k,180400.000000,4493,40,2400,남자,전라남도 나주시 그린로 317-7 (빛가람동),2019-02-14 오후 2:08,2019-01-09 오후 1:48,블로그광고,PC,2019-01-09,개인,일반
5,1004714862@k,54055.000000,4494,40,0,남자,세종특별자치시 마음로 181 (고운동),2019-04-15 오후 5:21,2019-04-12 오후 4:39,SNS광고,모바일,2019-01-09,개인,일반
6,10055065@n,50120.000000,3273,40,3280,남자,대구광역시 중구 국채보상로139길 40 (동인동3가),2018-10-22 오후 4:00,2018-10-22 오후 4:31,블로그광고,모바일,2018-10-22,개인,일반
7,1005540326292754@f,30270.000000,2778,40,1220,남자,대구광역시 중구 중앙대로 391 (동성로3가),2018-08-19 오후 6:06,2018-08-19 오후 6:11,SNS광고,모바일,2018-08-19,개인,일반
8,1005861444@k,75023.333333,4538,40,1800,남자,전라남도 광양시 중마로 542 (중동),2019-01-11 오후 1:14,2019-01-11 오후 3:02,SNS광고,모바일,2019-01-11,개인,일반
9,1005958854@k,67620.000000,4539,40,0,남자,경상북도 포항시 남구 구룡포읍 호미로 39,2019-04-30 오후 1:01,2019-04-30 오후 1:03,SNS광고,모바일,2019-01-11,개인,일반


In [99]:
df_result1["회원 가입일"]

0       2019-05-09
1       2019-01-07
2       2018-10-31
3       2019-01-09
4       2019-01-09
           ...    
4939    2018-10-30
4940    2019-03-29
4941    2018-11-01
4942    2018-10-16
4943    2018-01-03
Name: 회원 가입일, Length: 4944, dtype: object

In [105]:
df_result1['회원가입일(clean)'] = pd.to_datetime(df_result1['회원 가입일'])
df_result1['가입요일'] = df_result1['회원가입일(clean)'].dt.day_name()
df_result1.head(2)

,ID,상품구매금액,Unnamed: 0,나이,사용가능 적립금,성별,주소(동/읍/면),최종접속일,최종주문일,유입경로,유입기기,회원 가입일,사업자구분,회원구분,회원가입일(clean),가입요일
0,100304734@n,103300.000000,6308,40,100,남자,주소없음,2019-05-09 오후 2:09,2019-05-09 오후 2:12,SNS광고,모바일,2019-05-09,개인,일반,2019-05-09,Thursday
1,1003409866@k,95641.777778,4442,20,55900,남자,서울특별시 강남구 광평로51길 27 (수서동),2019-05-28 오후 4:40,2019-05-28 오후 4:41,카카오톡광고,모바일,2019-01-07,개인,일반,2019-01-07,Monday


In [118]:
df_result1.columns

Index(['ID', '상품구매금액', 'Unnamed: 0', '나이', '사용가능 적립금', '성별', '주소(동/읍/면)',
       '최종접속일', '최종주문일', '유입경로', '유입기기', '회원 가입일', '사업자구분', '회원구분',
       '회원가입일(clean)', '가입요일'],
      dtype='object')

In [131]:
df_x=df_result1[["나이","사용가능 적립금","성별","유입경로","회원 가입일","회원구분","가입요일","유입기기"]]

In [143]:
df_result1_dummy=pd.get_dummies(df_x)
df_result1_dummy.head()

,사용가능 적립금,나이_20,나이_30,나이_40,나이_50,나이_60,나이_70,나이_80,나이_90대,성별_남자,...,회원구분_특별회원,가입요일_Friday,가입요일_Monday,가입요일_Saturday,가입요일_Sunday,가입요일_Thursday,가입요일_Tuesday,가입요일_Wednesday,유입기기_PC,유입기기_모바일
0,100,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
1,55900,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2,1000,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
3,1300,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
4,2400,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0


In [133]:
#목표변수, 설명변수 데이터 지정
df_y=df_result1["상품구매금액"]
df_X=df_result1_dummy

In [134]:
df_y.shape

(4944,)

In [135]:
df_X.shape

(4944, 737)

In [136]:
df_train_x,df_test_x,df_train_y,df_test_y=train_test_split(df_X,df_y,test_size=0.3,random_state=1234)
print("train data X size :{}".format(df_train_x.shape))
print("train data Y size :{}".format(df_train_y.shape))
print("test data X size :{}".format(df_test_x.shape))
print("test data Y size :{}".format(df_test_y.shape))

train data X size :(3460, 737)
train data Y size :(3460,)
test data X size :(1484, 737)
test data Y size :(1484,)


In [ ]:

estimator=GradientBoostingRegressor(random_state=1234)
#구하고자 하는 parameter와 범위
param_grid={"criterion":["mse","friedman_mse","mae"],"max_features":["auto","sqrt","log2"],"learning_rate":[lr*0.1 for lr in range(1,11)],
           "max_depth":[depth for depth in range(1,11)],"min_samples_leaf":[n_leaf *1 for n_leaf in [1,5,8,11,15,18,20]]}

#설명력이 높은 최적 parameter 찾기
grid_dt=GridSearchCV(estimator,param_grid,scoring="r2",n_jobs=-1)
grid_dt.fit(df_train_x,df_train_y)

print("best estimator model:\n{}".format(grid_dt.best_estimator_))
print("\nbest parameter:\n{}".format(grid_dt.best_params_))
print("\nbest score:\n{}".format(grid_dt.best_score_.round(3)))

In [137]:
from sklearn.tree import DecisionTreeRegressor
estimator=DecisionTreeRegressor()
#구하고자 하는 parameter와 범위
param_grid={"max_depth":[depth for depth in [1,3,5,10]],
            "min_samples_split":[n_split*2 for n_split in [2,6,12,16,20]],
            "min_samples_leaf":[n_leaf * 1 for n_leaf in [1,5,15,20]],
           "criterion":['mse','mae','friedman_mse']}
#설명력이 높은 최적 parameter 찾기
grid_dt=GridSearchCV(estimator,param_grid,scoring="r2",n_jobs=-1,cv=5)
grid_dt.fit(df_train_x,df_train_y)

print("best estimator model:\n{}".format(grid_dt.best_estimator_))
print("\nbest parameter:\n{}".format(grid_dt.best_params_))
print("\nbest score:\n{}".format(grid_dt.best_score_.round(3)))

best estimator model:
DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=1,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=15, min_samples_split=40,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

best parameter:
{'criterion': 'mse', 'max_depth': 1, 'min_samples_leaf': 15, 'min_samples_split': 40}

best score:
0.021


In [138]:
tree_final = DecisionTreeRegressor(min_samples_leaf = 1,min_samples_split = 4,max_depth = 10,random_state = 1234,)
tree_final.fit(df_train_x,df_train_y)

In [139]:
# Train 데이터 설명력
print("Score on training set:{:.3f}".format(tree_final.score(df_train_x,df_train_y)))
# Validation 데이터 설명력
print("Score on Test set:{:.3f}".format(tree_final.score(df_test_x,df_test_y)))

In [140]:
# grid_dt.best_estimator_

In [141]:
# v_feature_name = df_train_x.columns
# df_importance = pd.DataFrame()
# df_importance["Feature"] = v_feature_name
# df_importance["Importance"] = tree_final.feature_importances_


# df_importance.sort_values("Importance",ascending = False,inplace = True)
# df_importance.round(3)